# Saving and Reapplying Clustering Results

This notebook demonstrates the **cluster once, apply many times** workflow.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

import tsam
from tsam import ClusterConfig, ClusteringResult

raw = pd.read_csv("testdata.csv", index_col=0)  # 4 columns: GHI, T, Wind, Load

## Two Paths to Aggregate

**Path A:** Cluster on ALL variables  
**Path B:** Cluster on WIND only, then apply to all variables

In [ ]:
# Path A: Cluster on ALL variables
result_all = tsam.aggregate(
    raw, n_periods=8, cluster=ClusterConfig(method="hierarchical")
)

# Path B: Cluster on WIND only, then transfer to all
result_wind = tsam.aggregate(
    raw[["Wind"]], n_periods=8, cluster=ClusterConfig(method="hierarchical")
)
result_transferred = result_wind.clustering.apply(raw)

## Comparing Cluster Assignments

- **Row 1** (all variables) differs from **Row 2** (wind only)
- **Row 2** and **Row 3** are **identical** - the transfer preserves the clustering!

In [ ]:
px.imshow(
    pd.DataFrame(
        {
            "Clustered on ALL variables": result_all.cluster_assignments,
            "Clustered on WIND only": result_wind.cluster_assignments,
            "Transferred to all variables": result_transferred.cluster_assignments,
        },
        index=pd.RangeIndex(
            start=0, stop=len(result_all.cluster_assignments), name="Original Period"
        ),
    ).T,
    color_continuous_scale="viridis",
    title="Cluster Assignments: Row 2 and 3 are IDENTICAL!",
)

## Verifying the Transfer

The wind typical periods are **identical** - but now we also have GHI, T, and Load:

In [ ]:
# Wind typical periods: wind-only vs transferred
wind_only = result_wind.typical_periods["Wind"]
wind_transferred = result_transferred.typical_periods["Wind"]

print(
    "Wind typical periods IDENTICAL after transfer:", wind_only.equals(wind_transferred)
)
print(f"Max difference: {(wind_only - wind_transferred).abs().max()}")

In [ ]:
# Verify: cluster assignments are equal
print(
    "Cluster assignments equal:",
    np.array_equal(
        result_wind.cluster_assignments, result_transferred.cluster_assignments
    ),
)

In [ ]:
# But now we have ALL columns!
print("Wind-only result columns:", result_wind.typical_periods.columns.tolist())
print(
    "Transferred result columns:", result_transferred.typical_periods.columns.tolist()
)

## Use Case: Save and Reload Clustering

In [ ]:
# Save clustering to file
result_wind.clustering.to_json("clustering.json")

# Later: load and apply to any data
clustering = ClusteringResult.from_json("clustering.json")
result_reloaded = clustering.apply(raw)

print(
    "Reloaded result identical:",
    result_transferred.typical_periods.equals(result_reloaded.typical_periods),
)

## Summary

```python
# Cluster on subset
result_wind = tsam.aggregate(data[["Wind"]], n_periods=8)

# Apply to all variables - wind stays identical!
result_all = result_wind.clustering.apply(data)

# Save for later
result_wind.clustering.to_json("clustering.json")
```